In [44]:
%load_ext autoreload
%autoreload 2

import time
import networkx as nx
import numpy as np
import logging
from copy import deepcopy
from collections import namedtuple
import torch
import pyro

from spatial_scene_grammars.nodes import *
from spatial_scene_grammars.rules import *
from spatial_scene_grammars.scene_grammar import *
from spatial_scene_grammars.parsing import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
from spatial_scene_grammars_examples.singles_pairs.grammar_constituency import *
#from spatial_scene_grammars_examples.singles_pairs.grammar_dependency import *
pyro.set_rng_seed(42)

grammar = SpatialSceneGrammar(
    root_node_type = Root,
    root_node_tf = torch.eye(4)
)
ground_truth_tree = grammar.sample_tree(detach=True)
observed_nodes = ground_truth_tree.get_observed_nodes()
print("Observed %d objects" % len(observed_nodes))
print("Objects: ", observed_nodes)

Observed 8 objects
Objects:  [<spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Root object at 0x7f29be40ab38>, <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Object object at 0x7f29be59b160>, <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Object object at 0x7f29be59b9e8>, <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Object object at 0x7f29be576ac8>, <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Object object at 0x7f29be576550>, <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Object object at 0x7f29be576e10>, <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Object object at 0x7f29be576470>, <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Object object at 0x7f29be5766d8>]


In [46]:
# Build up a big set of candidate intermediate nodes with both
# top-down and bottom-up generation.

max_recursion_depth = 10

top_down_candidate_intermediate_nodes = generate_top_down_intermediate_nodes_by_supertree(
    grammar, observed_nodes, max_recursion_depth=max_recursion_depth
)
bottom_up_candidate_intermediate_nodes = generate_bottom_up_intermediate_nodes_by_inverting_rules(
    grammar, observed_nodes
)
print("%d top-down, %d bottom-up candidates." %
      (len(top_down_candidate_intermediate_nodes),
       len(bottom_up_candidate_intermediate_nodes)))
candidate_intermediate_nodes = top_down_candidate_intermediate_nodes + bottom_up_candidate_intermediate_nodes
assert all([not node.observed for node in candidate_intermediate_nodes])

Candidate <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Singles object at 0x7f29be587e10> with pose tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])
Candidate <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Singles object at 0x7f29be5764e0> with pose tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])
Candidate <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Singles object at 0x7f29be80a128> with pose tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])
Candidate <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Singles object at 0x7f29c7db0f98> with pose tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])
Candidate <spatial_scene_grammars_examples.singles_pairs.grammar_constituency.Singles object

In [59]:
parse_trees = infer_mle_tree_with_mip_from_proposals(
    grammar, observed_nodes, candidate_intermediate_nodes, verbose=True, N_solutions=1
)

Optimization success?:  True
Logfile: 

Gurobi 9.0.2 (linux64) logging started Tue Nov 30 16:28:08 2021

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 189 rows, 473 columns and 996 nonzeros
Model fingerprint: 0xf76bcae2
Variable types: 0 continuous, 473 integer (473 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 8e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 108 rows and 107 columns
Presolve time: 0.01s
Presolved: 81 rows, 366 columns, 838 nonzeros
Variable types: 0 continuous, 366 integer (366 binary)

Root relaxation: objective 6.811424e+02, 137 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  681.14242    0    7          -  681.14242      -     -    0s
H    0     0                     681.3592150  681.14242  0.03%     -  